In [ ]:
# Import used packages
%matplotlib notebook
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from matplotlib import animation
import random
import copy

# Network implementation

In [ ]:
# Create positions in rectangle of xLen x yLen
def randompos(nodes, xLen, yLen):
    pos = []
    for i in range(nodes):
        pos.append([xLen*random.uniform(0, 1), yLen*random.uniform(0, 1)])
    return pos

# Link nodes using KDtrees
def linknodesKDtree(pos, radius):
    treePos = spatial.KDTree(pos)
    links = treePos.query_ball_tree(other= treePos, r= radius)
    for i in range(len(links)):
        links[i].remove(i)
    return links

# Create network in a dictionary structure
def createnetwork(nodes, radius, xLen, yLen):
    Network = {'nodes': nodes, 'pos': randompos(nodes, xLen, yLen)}
    Network['links'] = linknodesKDtree(Network['pos'], radius)
    return Network

# plot nodes
def plotnodes(pos, ax, sizeNodes, colorNodes):
    ax.scatter([item[0] for item in pos], [item[1] for item in pos], s= sizeNodes, c= colorNodes, zorder= 2)
    
#plot edges
def plotedges(pos, ax, links, colorEdges, alphaEdges):
    xList = []
    yList = []
    for i in range(len(links)):
        for node in links[i]:
            if node > i:
                xList += [pos[i][0]]
                yList += [pos[i][1]]
                xList += [pos[node][0]]
                yList += [pos[node][1]]
                xList += [np.nan]
                yList += [np.nan]
    ax.plot(xList, yList, color = colorEdges, alpha= alphaEdges, zorder= 1)

# Implementation of SEIR model
## Model
The SEIR model describes the propagation of a disease in 4 stages: susceptible, exposed, infected and recovered. Each state has different properties. At the start most people are susceptible, these can get exposed when in contact with someone who is infected. Exposed people cannot infect other people yet. They will get infectious after some time, dependent on the incubation period. Infected people can recover, when recovered they are immune to the dissease.
## Parameters
We used $\beta$ as the probability to become exposed when in contact with 1 infected person, $\delta$ is the probability of becoming infected when exposed and $\gamma$ is the probability to recover when infected.
## Datastructure
To contain all the data, we used a dictionairy 'Network'. This contains the following keys: <br>
- 'nodes': int, the amount of nodes <br>
- 'pos': list, list of lists of length 2 containing the x and y positions of the nodes <br>
- 'links': list, list of lists containing the numbers corresponding to the nodes connected to the i'th node <br>
- 'currentState': list, list of the stage of the i'th node at the current timestep <br>
- 'currentColor': list, list of the color of the i'th node at the current timestep <br>
- 'newState': list, list of the stage of the i'th node at the next timestep <br>
- 'dataStates': list (of lists), list of all the lists passing 'currentState' <br>
- 'dataColors': list (of lists), list of all the lists passing 'currentColor' <br>
- 'time': list, list of consecutive integers representing the timesteps <br>
- 'S': list, list of how many nodes are in the susceptible stage at the i'th timestep <br>
- 'E': list, list of how many nodes are in the exposed stage at the i'th timestep <br>
- 'I': list, list of how many nodes are in the infected stage at the i'th timestep <br>
- 'R': list, list of how many nodes are in the recovered stage at the i'th timestep <br>
- 'socialDistanceTimestep': int, the timestep at which social distancing occurs (defaults to np.nan)

In [ ]:
# Declare global parameters
class ST_base():
    pass

ST = ST_base()
ST.S = 0
ST.E = 1
ST.I = 2
ST.R = 3

class ST_color():
    pass

STcolor = ST_color()
STcolor.S = 'palegreen'
STcolor.E = 'orchid'
STcolor.I = 'tomato'
STcolor.R = 'deepskyblue'

class PL_attributes():
    pass

PL = PL_attributes()
PL.edgeColor = 'grey'
PL.edgeAlpha = 0.4
PL.nodeSize = 80
PL.zoom = 5

In [ ]:
def start(Network, exposedPerc, local):
    Network['currentState'] = [ST.S]*Network['nodes']
    Network['currentColor'] = [STcolor.S]*Network['nodes']
    Network['socialDistanceTimestep'] = np.nan
    exposedNumber = exposedPerc*Network['nodes']
    if local:
        exposed = startperc(Network, exposedNumber)
    else:
        exposed = []
        while i < exposedNumber:
            randomN = random.randint(0, Network['nodes'] - 1)
            if randomN not in exposed:
                exposed.append(randiomN)
                i += 1
    for i in exposed:
            Network['currentState'][i] = ST.E
            Network['currentColor'][i] = STcolor.E
            
    return Network

def startperc(Network, exposedNumber):
    exposed = [random.randint(0, Network['nodes'] - 1)]
    while len(exposed) < exposedNumber:
        exposedNew = exposed.copy()
        for i in exposedNew:
            for j in range(len(Network['links'][i])):
                if len(exposedNew) < exposedNumber:
                    if Network['links'][i][j] not in exposedNew:
                        exposedNew.append(Network['links'][i][j])
                else:
                    return exposedNew
        if len(exposed) == len(exposedNew):
            return startperc(Network, exposedNumber)
        exposed = exposedNew
    
    return exposed

In [ ]:
# Create list of the next state after 1 timestep
def updateSEIR(Network, beta, delta, gamma):
    Network['newState'] = []
    for i in range(Network['nodes']):
        if Network['currentState'][i] == ST.S:
            Phealthy = 1
            for j in Network['links'][i]:
                if Network['currentState'][j] == ST.I:
                    Phealthy *= (1 - beta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                Network['newState'].append(ST.E)
                Network['currentColor'][i] = STcolor.E
            else:
                Network['newState'].append(Network['currentState'][i])
        elif Network['currentState'][i] == ST.E:
            randomN = random.uniform(0, 1)
            if randomN < delta:
                Network['newState'].append(ST.I)
                Network['currentColor'][i] = STcolor.I
            else:
                Network['newState'].append(Network['currentState'][i])
        elif Network['currentState'][i] == ST.I:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                Network['newState'].append(ST.R)
                Network['currentColor'][i] = STcolor.R
            else:
                Network['newState'].append(Network['currentState'][i])
        else:
            Network['newState'].append(Network['currentState'][i])
    Network['currentState'] = Network['newState']
    
    return Network

# Creates list holding all information of the situation in every time step 
# and plots color-coded network of every timestep if plotNetwork is True
def spreaddiseaseSEIR(Network, timesteps, beta, delta, gamma, xLen, yLen, plotNetworks, socialDistance = False, socialDistancePerc = 0, removePerc = 0):
    Network['dataStates'] = [Network['currentState']]
    Network['dataColors'] = [Network['currentColor'].copy()]
    Network = countingSEIR(Network)
    
    if plotNetworks:
            for i in range(timesteps):
                Network = updateSEIR(Network, beta, delta, gamma)
                Network['dataStates'].append(Network['currentState'])
                Network['dataColors'].append(Network['currentColor'].copy())
                Network = countingSEIR(Network)

                if socialDistance:
                    if Network['I'][-1] >= Network['nodes']*socialDistancePerc:
                        Network['links'] = removeedges(Network['links'], removePerc)
                        Network['socialDistanceTimestep'] = i
                        socialDistance = False
                
                fig, ax = plt.subplots(figsize = [6*xLen, 6*yLen])
                plotedges(Network['pos'], ax, Network['links'], PL.edgeColor, PL.edgeAlpha)
                plotnodes(Network['pos'], ax, PL.nodeSize, Network['currentColor'])
                plt.axis('off')
                plt.show()
    
    else:
        for i in range(timesteps):
            Network = updateSEIR(Network, beta, delta, gamma)
            Network['dataStates'].append(Network['currentState'])
            Network['dataColors'].append(Network['currentColor'].copy())
            Network = countingSEIR(Network)
            
            if socialDistance:
                if Network['I'][-1] >= Network['nodes']*socialDistancePerc:
                    Network['links'] = removeedges(Network['links'], removePerc)
                    Network['socialDistanceTimestep'] = i
                    socialDistance = False
    
    return Network

# Count the amount of people in each state for every timestep
def countingSEIR(Network):
    counting = np.bincount(Network['currentState'], minlength = 4)
    Network['S'].append(counting[ST.S])
    Network['E'].append(counting[ST.E])
    Network['I'].append(counting[ST.I])
    Network['R'].append(counting[ST.R])
        
    return Network

# Executes all the functions above 
# and plots the evolution in all states with respect to time if plotStates == True
def everythingSEIR(Network, timesteps, beta, delta, gamma, xLen, yLen, plotNetworks, plotStates, socialDistance = False, socialDistancePerc = 0, removePerc = 0):
    Network['S'] = []
    Network['E'] = []
    Network['I'] = []
    Network['R'] = []
    
    if plotNetworks:
        fig, ax = plt.subplots(figsize = [PL.zoom*xlen, PL.zoom*ylen])
        plotedges(Network['pos'], ax, Network['links'], PL.edgeColor, PL.edgeAlpha)
        plotnodes(Network['pos'], ax, PL.nodeSize, Network['currentColor'])
        plt.axis('off')
        plt.show()
    
    Network = spreaddiseaseSEIR(Network, timesteps, beta, delta, gamma, xLen, yLen, plotNetworks, socialDistance, socialDistancePerc, removePerc)
    
    Network['time'] = np.arange(0, timesteps + 1, 1)
    if plotStates:
        figData = plt.figure()

        plt.plot(Network['time'], Network['S'], label = 'S', color= STcolor.S)
        plt.plot(Network['time'], Network['E'], label = 'E', color= STcolor.E)
        plt.plot(Network['time'], Network['I'], label = 'I', color= STcolor.I)
        plt.plot(Network['time'], Network['R'], label = 'R', color= STcolor.R)
        plt.axvline(Network['socialDistanceTimestep'])
                
        plt.title('', size= 20)
        plt.ylabel('', size= 20)
        plt.xlabel('', size= 20)
        plt.legend(fontsize= 12)
        plt.ylim(0, nodes)
        plt.xlim(0, timesteps)
        plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
        plt.grid()

        plt.show()
    return Network

# Averages over N different networks and evolutions of disseasespreading
# and plots the average evolution of all states if plotAverage == True
def averageSEIR(nodes, exposedPerc, radius, timesteps, local, beta, delta, gamma, xLen, yLen, N, plotAverage):
    averageS = np.zeros(timesteps + 1)
    averageE = np.zeros(timesteps + 1)
    averageI = np.zeros(timesteps + 1)
    averageR = np.zeros(timesteps + 1)
    for i in range(N):
        Network = createnetwork(nodes, radius, xLen, yLen)
        Network = start(Network, exposedPerc, local)
        Network = everythingSEIR(Network, timesteps, beta, delta, gamma, xLen, yLen, plotNetworks= False, plotStates= False)
        averageS += Network['S']
        averageE += Network['E']
        averageI += Network['I']
        averageR += Network['R']
    
    averageS = averageS/N
    averageE = averageE/N
    averageI = averageI/N
    averageR = averageR/N
    
    if plotAverage:
        figAverage = plt.figure()

        plt.plot(Network['time'], averageS, label = 'S', color= STcolor.S)
        plt.plot(Network['time'], averageE, label = 'E', color= STcolor.E)
        plt.plot(Network['time'], averageI, label = 'I', color= STcolor.I)
        plt.plot(Network['time'], averageR, label = 'R', color= STcolor.R)

        plt.title('', size= 20)
        plt.ylabel('', size= 20)
        plt.xlabel('', size= 20)
        plt.legend(fontsize= 12)
        plt.ylim(0, nodes)
        plt.xlim(0, timesteps)
        plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
        plt.grid()

        plt.show()
    
    return averageS, averageE, averageI, averageR

In [ ]:
def removeedges(links, removePerc):
    removeN = round(sum(map(len, links))*removePerc/2)
    for i in range(removeN):
        randomNode = random.choice(random.choices(links, weights= map(len, links), k= 1)[0])
        randomConnectedNode = random.choice(links[randomNode])
        links[randomNode].remove(randomConnectedNode)
        links[randomConnectedNode].remove(randomNode)
    return links

In [ ]:
# Create animation
nodes = 200
radius = 0.12
xLen = 1
yLen = 1
exposedPerc = 0.1
local = True
beta = 0.5
delta = 0.3
gamma = 0.1
timesteps = 30

interval = 400
plotStates= False

fig, ax = plt.subplots(figsize = [PL.zoom*xLen, PL.zoom*yLen])
Network = createnetwork(nodes, radius, xLen, yLen)
Network = start(Network, exposedPerc, local)
Network = everythingSEIR(Network, timesteps, beta, delta, gamma, xLen, yLen, plotNetworks= False, plotStates= plotStates)
plotedges(Network['pos'], ax, Network['links'], PL.edgeColor, PL.edgeAlpha)
def animate(i):
    ax.axis('off')
    plotnodes(Network['pos'], ax, PL.nodeSize, Network['dataColors'][i])
    ax.set_title(i)
aniSEIR = animation.FuncAnimation(fig, animate, frames= timesteps + 1, repeat= True, interval= interval)
plt.show()

In [ ]:
# Average over N
nodes = 200
exposedPerc = 0.1
radius = 0.12
local = True
beta = 0.6
delta = 0.3
gamma = 0.1
xLen = 1
yLen = 1
timesteps = 100

N = 100
plotAverage = True

averageS, averageE, averageI, averageR = averageSEIR(nodes, exposedPerc, radius, timesteps, local, beta, delta, gamma, xLen, yLen, N, plotAverage)

In [ ]:
# Remove amount of edges after certain infection percent
nodes = 200
exposedPerc = 0.1
radius = 0.12
local = True
beta = 0.6
delta = 0.4
gamma = 0.02
xLen = 1
yLen = 1
timesteps = 100

interval = 400
plotStates= True

socialDistance= True
socialDistancePerc = 0.3
removePerc = 0.7

fig, ax = plt.subplots(figsize = [PL.zoom*xLen, PL.zoom*yLen])
Network = createnetwork(nodes, radius, xLen, yLen)
Network = start(Network, exposedPerc, local)
startNetwork = copy.deepcopy(Network)
Network = everythingSEIR(Network, timesteps, beta, delta, gamma, xLen, yLen, plotNetworks= False, plotStates= plotStates, socialDistance= socialDistance, socialDistancePerc= socialDistancePerc, removePerc= removePerc)
def animate(i):
    if i == 0:
        ax.cla()
        ax.axis('off')
        plotedges(startNetwork['pos'], ax, startNetwork['links'], PL.edgeColor, PL.edgeAlpha)
        
    if i == Network['socialDistanceTimestep']:
        ax.cla()
        ax.axis('off')
        plotedges(Network['pos'], ax, Network['links'], PL.edgeColor, PL.edgeAlpha)
        
    plotnodes(Network['pos'], ax, PL.nodeSize, Network['dataColors'][i])
    ax.set_title(i)
    
aniRemoved = animation.FuncAnimation(fig, animate, frames= timesteps + 1, repeat= True, interval= interval)
#aniRemoved.save('aniRemoved.gif')
plt.show()

In [ ]:
# Remove amount of edges and compare (keep this code for when we want to compare the social distancing visually)
'''nodes = 100
exposedPerc = 0.1
radius = 0.15
beta = 0.7
delta = 0.5
gamma = 0.02
pictureWidth = 1
pictureHeight = 1
timesteps = 30

interval = 400
plotStates= False

socialDistancePerc = 0.3
removePerc = 0.3

figedges, (ax1, ax2) = plt.subplots(nrows= 1, ncols= 2, figsize = [2*5*pictureWidth, 5*pictureHeight])
Network = start(nodes, radius, exposedPerc, pictureWidth, pictureHeight)
startNetwork = copy.deepcopy(Network)
Network = everythingSEIR(Network, timesteps, beta, delta, gamma, pictureWidth, pictureHeight, plotNetworks= False, plotStates= plotStates, socialDistance= True, socialDistancePerc= socialDistancePerc, removePerc= removePerc)

def animateedges(i):
    if i == 0:
        ax1.cla()
        ax2.cla()
        plotedges(Network['pos'], ax1, startNetwork['z'], PL.edgeColor, PL.edgeAlpha)
        plotedges(Network['pos'], ax2, Network['sets'], PL.edgeColor, PL.edgeAlpha)
        ax1.axis('off')
        ax2.axis('off')
    try:
        if i == Network['socialDistanceTimestep']:
            ax1.cla()
            ax1.axis('off')
            plotedges(Network['pos'], ax1, Network['sets'], PL.edgeColor, PL.edgeAlpha)
    except:
        np.nan
    plotnodes(startNetwork['pos'], ax1, PL.nodeSize, Network['dataColors'][i])
    plotnodes(Network['pos'], ax2, PL.nodeSize, Network['dataColors'][i])
    ax1.set_title(i)
    ax2.set_title(i)
    
aniRemoved = animation.FuncAnimation(figedges, animateedges, frames= timesteps + 1, repeat= True, interval= interval)
#aniRemoved.save('aniRemoved.gif')
plt.show()'''